# SQL Generation and Analysis Workflow Test

This notebook demonstrates the complete workflow of:
1. Generating SQL from natural language
2. Refining the generated SQL
3. Executing the refined SQL
4. Analyzing the results

Let's test each component sequentially.

In [1]:
# Test query
user_query = "Find the name and email of a learner named David Gardner or who has the email allisonthomson@example.com"

## 1. SQL Generation Test

In [2]:
from engine.generator import SQLGenerator

def test_generator():
    """Test SQL generation functionality"""
    generator = SQLGenerator()

    print(f"\nQuery: '{user_query}'")
    
    try:
        # Generate SQL
        print("\nGenerating SQL...")
        results = generator.main_generator(user_query)
        
        print("\nSchema Information:")
        print(results['formatted_metadata'])
        
        print("\nGenerated SQL:")
        print(results['generated_sql'])
        
        return results['generated_sql']
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

generated_sql = test_generator()


Query: 'Find the name and email of a learner named David Gardner or who has the email allisonthomson@example.com'

Generating SQL...

Schema Information:
Country (id, name)
State (id, name, code, country_code)
District (id, name, code, state_code)
Institution (id, name, short_name, aicte_code, eamcet_code, l4g_code, l4g_group_code, type, address, website, latlong, district_code)
Degree (id, name, short_name)
Branch (id, name, short_name, degree_code)
Department (id, name, type)
Designation (id, name, type, priority)
Knowledge_Partner (id, name, address, website, info)
Course (id, name, info, knowledge_partner_code)
Module (id, name, info, module_sequence_number, theory_practical, duration_minutes, course_code)
Specialization (id, name, info, knowledge_partner_code)
Specialization_Course (id, course_sequence_number, course_code, specialization_code)
Program (id, name, info, knowledge_partner_code)
Program_Specialization (id, program_code, specialization_code)
Program_Requirement (id, n

## 2. SQL Refinement Test

In [3]:
from engine.refiner import SQLRefiner

def test_refiner(sql_query):
    """Test SQL refinement functionality"""
    refiner = SQLRefiner()
    
    print(f"\nInput SQL: '{sql_query}'")
    
    try:
        # Refine SQL
        print("\nRefining SQL...")
        results = refiner.main_refiner(sql_query)
        
        print("\nExtracted Entities:")
        if results['extracted_entities']:
            for entity in results['extracted_entities']:
                print(f"Table: {entity['table']}")
                print(f"Column: {entity['column']}")
                print(f"Value: {entity['value']}")
                print("---")
        else:
            print("No entities were extracted from the SQL query")
        
        print("\nValue Mappings:")
        if results['value_mappings']:
            for mapping in results['value_mappings']:
                print(f"Original: '{mapping['original_value']}'")
                print(f"Matched: '{mapping['matched_value']}'")
                print(f"Score: {mapping['score']}")
                print("---")
        else:
            print("No value mappings needed")
        
        print("\nRefined SQL:")
        if results['refined_sql'] != results['original_sql']:
            print("SQL was refined to:")
            print(results['refined_sql'])
        else:
            print("No refinements were needed. Using original SQL.")
            
        return results['refined_sql']
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

if generated_sql:
    refined_sql = test_refiner(generated_sql)
else:
    print("Skipping refinement as no SQL was generated")


Input SQL: ' SELECT Learner.name, Learner.email
   FROM Learner
   WHERE Learner.name = 'David Gardner'
   UNION ALL
   SELECT Learner.name, Learner.email
   FROM Learner
   INNER JOIN Learner_Education ON Learner.id = Learner_Educition.learner_code
   WHERE Learner_education.email = 'allisonthomson@example.com';'

Refining SQL...

Extracted Entities:
Table: Learner
Column: name
Value: David Gardner
---
Table: Learner
Column: email
Value: 'allisonthomson@example.com'
---

Value Mappings:
Original: 'David Gardner'
Matched: 'David Garner'
Score: 96
---
Original: ''allisonthomson@example.com''
Matched: 'allisonthompson@example.com'
Score: 98
---

Refined SQL:
SQL was refined to:
 SELECT Learner.name, Learner.email
   FROM Learner
   WHERE Learner.name = 'David Garner'
   UNION ALL
   SELECT Learner.name, Learner.email
   FROM Learner
   INNER JOIN Learner_Education ON Learner.id = Learner_Educition.learner_code
   WHERE Learner_education.email = 'allisonthompson@example.com';


## 3. SQL Execution Test

In [4]:
from engine.executor import SQLExecutor

def test_executor(sql_query):
    """Test SQLExecutor functionality"""
    executor = SQLExecutor()

    print(f"Executing Query:\n{sql_query}")
    
    success, results, formatted_results, error = executor.main_executor(sql_query)
    
    if success:
        print(f"\nSuccess! Found {len(results)} rows")
        print("\nFormatted Results:")
        print(formatted_results)
        print("\nRaw Results:")
        print(results)
        return results
    else:
        print(f"\nFailed: {error}")
        return None

if refined_sql:
    execution_results = test_executor(refined_sql)
else:
    print("Skipping execution as no refined SQL was generated")

Executing Query:
 SELECT Learner.name, Learner.email
   FROM Learner
   WHERE Learner.name = 'David Garner'
   UNION ALL
   SELECT Learner.name, Learner.email
   FROM Learner
   INNER JOIN Learner_Education ON Learner.id = Learner_Educition.learner_code
   WHERE Learner_education.email = 'allisonthompson@example.com';

Failed: no such column: Learner_education.email


## 4. Results Analysis Test

In [5]:
from engine.analyzer import SQLAnalyzer

def test_analyzer(query, results):
    """Test SQLAnalyzer functionality"""
    analyzer = SQLAnalyzer()
    
    # Test analysis
    analysis_results = analyzer.main_analyzer(query, results)
    
    print("\nAnalysis Results:")
    if analysis_results['success']:
        print(f"Query: {analysis_results['query_info']}")
        print(f"Record count: {analysis_results['record_count']}")
        print(f"\nSuccess: {analysis_results['success']}")
        print("\nAnalysis:")
        print(analysis_results['analysis'])
    else:
        print(f"Analysis failed: {analysis_results['error']}")

if execution_results:
    test_analyzer(user_query, execution_results)
else:
    print("Skipping analysis as no execution results were generated")

Skipping analysis as no execution results were generated


In [6]:
from llm_config.llm_call import generate_text

# Define a prompt to test the conversation history
prompt = "What was out last chat about?"
response = generate_text(prompt)
print("Generated Response:")
print(response)

Generated Response:
 The last chat revolved around the extraction of entities from a given SQL query and providing the entities in the table_name|column_name|comparison_value format as per requirements. The query was modified with the replacements you specified, replacing 'David Gardner' with 'David Garner' and 'allisonthomson@example.com' with 'allisonthompson@example.com'.
